<a href="https://colab.research.google.com/github/haaris519/Get-Started-with-Python-Google/blob/main/Finance_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime

# Initialize SQLite database
conn = sqlite3.connect("finance_tracker.db")
cursor = conn.cursor()

# Create table for transactions
cursor.execute("""
CREATE TABLE IF NOT EXISTS transactions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    category TEXT,
    amount REAL,
    type TEXT,  -- 'income' or 'expense'
    date TEXT
)
""")
conn.commit()

# Add a transaction with validation
def add_transaction():
    category = input("Enter category (e.g., Rent, Salary, Groceries): ").strip()

    # Validate amount input
    while True:
        try:
            amount = float(input("Enter amount: "))
            if amount <= 0:
                raise ValueError("Amount must be greater than zero.")
            break
        except ValueError as e:
            print(f"Invalid input: {e}. Please enter a valid positive number.")

    # Validate type input
    while True:
        t_type = input("Enter type (income/expense): ").strip().lower()
        if t_type in ['income', 'expense']:
            break
        else:
            print("Invalid type. Please enter 'income' or 'expense'.")

    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    cursor.execute("INSERT INTO transactions (category, amount, type, date) VALUES (?, ?, ?, ?)",
                   (category, amount, t_type, date))
    conn.commit()
    print("Transaction added successfully!\n")

# View summary of finances
def view_summary():
    cursor.execute("SELECT SUM(amount) FROM transactions WHERE type='income'")
    total_income = cursor.fetchone()[0] or 0

    cursor.execute("SELECT SUM(amount) FROM transactions WHERE type='expense'")
    total_expense = cursor.fetchone()[0] or 0

    balance = total_income - total_expense

    print("\n--- Financial Summary ---")
    print(f"Total Income: ${total_income:.2f}")
    print(f"Total Expenses: ${total_expense:.2f}")
    print(f"Balance: ${balance:.2f}\n")

# Visualize spending patterns
def visualize_data():
    cursor.execute("SELECT category, SUM(amount) FROM transactions WHERE type='expense' GROUP BY category")
    data = cursor.fetchall()

    if not data:
        print("No expense data to visualize.\n")
        return

    categories = [row[0] for row in data]
    amounts = [row[1] for row in data]

    # Bar Chart
    plt.figure(figsize=(10, 6))
    plt.bar(categories, amounts, color='skyblue')
    plt.title("Expenses by Category")
    plt.xlabel("Category")
    plt.ylabel("Amount")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Pie Chart
    plt.figure(figsize=(8, 8))
    plt.pie(amounts, labels=categories, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
    plt.title("Expense Distribution")
    plt.show()

# Delete a transaction by ID
def delete_transaction():
    view_transactions()
    try:
        trans_id = int(input("Enter the ID of the transaction to delete: "))
        cursor.execute("DELETE FROM transactions WHERE id=?", (trans_id,))
        conn.commit()
        print("Transaction deleted successfully!\n")
    except ValueError:
        print("Invalid ID. Please enter a valid number.\n")

# View all transactions
def view_transactions():
    cursor.execute("SELECT * FROM transactions")
    transactions = cursor.fetchall()

    if not transactions:
        print("No transactions found.\n")
        return

    print("\n--- All Transactions ---")
    print(f"{'ID':<5} {'Category':<15} {'Amount':<10} {'Type':<10} {'Date'}")
    print("-" * 50)
    for trans in transactions:
        print(f"{trans[0]:<5} {trans[1]:<15} {trans[2]:<10.2f} {trans[3]:<10} {trans[4]}")
    print()

# Main menu
def main():
    while True:
        print("\n--- Personal Finance Tracker ---")
        print("1. Add Transaction")
        print("2. View Summary")
        print("3. Visualize Expenses")
        print("4. View All Transactions")
        print("5. Delete a Transaction")
        print("6. Exit")

        choice = input("Enter your choice: ").strip()
        if choice == '1':
            add_transaction()
        elif choice == '2':
            view_summary()
        elif choice == '3':
            visualize_data()
        elif choice == '4':
            view_transactions()
        elif choice == '5':
            delete_transaction()
        elif choice == '6':
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.\n")

# Run the program
if __name__ == "__main__":
    main()



--- Personal Finance Tracker ---
1. Add Transaction
2. View Summary
3. Visualize Expenses
4. View All Transactions
5. Delete a Transaction
6. Exit
Enter your choice: 6
Exiting program. Goodbye!
